# GPU-Accelerated Data Science: Introduction to Blazing SQL and RAPIDS

Programming on GPUs can be intimidating. In the past it required a solid knowledge of C++ and CUDA, and the ability to think *in parallel*. Today, with [RAPIDS](https://rapids.ai) and [BlazingSQL](https://blazingsql.com), you can get started using the immense power of GPUs in no time. And all of that with minimal code changes: whether you use PyData ecosystem tools like pandas or Scikit-Learn, or are more familiar with SQL, RAPIDS and BlazingSQL will enable you to achieve immense speedups by offloading all the computations to GPU.

## Imports
First, of course, let's import the tools that we need.

In [ ]:
import cudf
import blazingsql as bsql
import s3fs
import numpy as np
from collections import OrderedDict
from IPython.display import HTML
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions, LabelSet
from bokeh.plotting import gmap

### `BlazingContext`
You must establish a `BlazingContext` to connect to a BlazingSQL instance to create tables, run queries, and basically do anything with BlazingSQL.

In [ ]:
bc = bsql.BlazingContext()

The `BlazingContext` is an entrypoint for all things Blazing. In this particular instance we start the `BlazingContext` with default parameters but there are many ways to customize it and expand its capablities.

|Argument|Required|Description|Defaults|
|:-------|:------:|:----------|-------:|
allocator|      No|Options are "default", "managed". Where "managed" uses Unified Virtual Memory (UVM) and may use system memory if GPU memory runs out, or "existing" where it assumes you have already set the rmm allocator and therefore does not initialize it (this is for advanced users.).|"managed"
dask_client|No|The dask client used for communicating with other nodes. This is only necessary for running BlazingSQL with multiple nodes.|None
enable_logging|No|If set to True the memory allocator logging will be enabled, but can negatively impact performance. This is for advanced users.|False
initial_pool_size|No|Initial size of memory pool in bytes (if pool=True). If None, it will default to using half of the GPU memory.|None
pool|No|If True, allocate a memory pool in the beginning. This can greatly improve performance.|False
network_interface|No|Network interface used for communicating with the dask-scheduler. See note below.|'eth0'
config_options|No|A dictionary for setting certain parameters in the engine.|

## Data reading and querying

There are two ways to load and query data using tools from the RAPIDS ecosystem: load directly into memory using `cudf` or `.create_table()` using `BlazingContext`.

### Flight data

In [ ]:
flight_data_path = 's3://bsql/data/air_transport/flight_ontime_2020-0[1-5].parquet'
s3 = s3fs.S3FileSystem(anon=True)
files = [f's3://{f}' for f in s3.glob(flight_data_path)]
files

#### cuDF

In [ ]:
%%time
flights = []

for f in files:
    flights.append(cudf.read_parquet(f, storage_options={'anon': True}))
    
flights = cudf.concat(flights)

In [ ]:
flights.head(5)

In [ ]:
print(f'Total number of flights in the dataset: {len(flights):,}')

#### BlazingSQL

In [ ]:
_ = bc.s3(
    'bsql'
    , bucket_name = 'bsql'
)

In [ ]:
bc.create_table('air_transport', files)

In [ ]:
%%time
bc.sql('SELECT * FROM air_transport LIMIT 5')

In [ ]:
print(f'Total number of flights in the dataset: {bc.sql("SELECT COUNT(*) AS CNT FROM air_transport")["CNT"].iloc[0]:,}')

#### Columns and data types

In [ ]:
flights.columns

In [ ]:
flights.dtypes

The `BlazingContext` returns a cuDF DataFrame object so we have access to the same API!

In [ ]:
bc_df = bc.sql('SELECT * FROM air_transport LIMIT 5')
type(bc_df)

In [ ]:
bc_df.columns

In [ ]:
bc_df.dtypes

### Airlines and airports data

In [ ]:
airports_path = 's3://bsql/data/air_transport/airports.csv'
airlines_path = 's3://bsql/data/air_transport/airlines.csv'

In [ ]:
airports_dtypes = OrderedDict([
      ('Airport ID', 'int64')
    , ('Name', 'str')
    , ('City', 'str')
    , ('Country', 'str')
    , ('IATA', 'str')
    , ('ICAO', 'str')
    , ('Latitude', 'float64')
    , ('Longitude', 'float64')
    , ('Altitude', 'int64')
    , ('Timezone', 'str')
    , ('DST', 'str')
    , ('Type', 'str')
    , ('Source', 'str')
])

airports = cudf.read_csv(
    airports_path
    , names=list(airports_dtypes.keys())
    , dtype=list(airports_dtypes.values())
    , storage_options={'anon': True}
)
airports.head()

In [ ]:
airlines_dtypes = OrderedDict([
    ('Airline ID', 'int64')
    , ('Name', 'str')
    , ('Alias', 'str')
    , ('IATA', 'str')
    , ('ICAO', 'str')
    , ('Callsign', 'str')
    , ('Country', 'str')
    , ('Active', 'str')
])

airlines = cudf.read_csv(
    airlines_path
    , names=list(airlines_dtypes.keys())
    , dtype=list(airlines_dtypes.values())
    , storage_options={'anon': True}
)
airlines.head()

You can create BlazingSQL tables directly from cuDF DataFrames.

In [ ]:
bc.create_table('airports', airports)
bc.create_table('airlines', airlines)

And now we can use it to query and join these datasets.

In [ ]:
%%time
bc.sql('''
    SELECT A.FL_DATE
        , A.OP_UNIQUE_CARRIER
        , B.Name AS CARRIER_NAME
        , A.ORIGIN
        , C.Name AS ORIGIN_NAME
        , C.City AS ORIGIN_CITY
        , A.DEST
        , D.Name AS DEST_NAME
        , D.City AS DEST_CITY
    FROM air_transport AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    LEFT OUTER JOIN airports AS C
        ON A.ORIGIN = C.IATA
    LEFT OUTER JOIN airports AS D
        ON A.DEST = D.IATA
    LIMIT 4
''')

The beauty of the ecosystem, and BlazingSQL in particular, comes from the direct inter-operability with RAPIDS: we can create tables from cudf and any file format supported by cuDF, either local or remote; you can register buckets from `s3` and `gcp` with the `BlazingContext` with support for Azure coming in future releases. So, as easily, we could simply create the tables directly from files and trivially write code that returns a cuDF DataFrame by joining Parquet and CSV files in just couple of lines!

In [ ]:
bc.create_table('airports_table', airports_path, names=list(airports_dtypes.keys()), dtype=list(airports_dtypes.values()))
bc.create_table('airlines_table', airlines_path, names=list(airlines_dtypes.keys()), dtype=list(airlines_dtypes.values()))

In [ ]:
%%time
bc.sql('''
    SELECT A.FL_DATE
        , A.OP_UNIQUE_CARRIER
        , B.Name AS CARRIER_NAME
        , A.ORIGIN
        , C.Name AS ORIGIN_NAME
        , C.City AS ORIGIN_CITY
        , A.DEST
        , D.Name AS DEST_NAME
        , D.City AS DEST_CITY
    FROM air_transport AS A                // READING FROM PARQUET
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    LEFT OUTER JOIN airports_table AS C    // READING FROM CSV
        ON A.ORIGIN = C.IATA
    LEFT OUTER JOIN airports_table AS D    // READING FROM CSV
        ON A.DEST = D.IATA
    LIMIT 4
''')

In [ ]:
%%time
(
    flights[['FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST']]
    .merge(airlines[['IATA', 'Name']], left_on='OP_UNIQUE_CARRIER', right_on='IATA')
    .rename(columns={'Name': 'CARRIER_NAME'})
    .drop(columns=['IATA'])
    .merge(airports[['IATA', 'Name', 'City']], left_on='ORIGIN', right_on='IATA')
    .rename(columns={'Name': 'ORIGIN_NAME', 'City': 'ORIGIN_CITY'})
    .drop(columns=['IATA'])
    .merge(airports[['IATA', 'Name', 'City']], left_on='DEST', right_on='IATA')
    .rename(columns={'Name': 'DEST_NAME', 'City': 'DEST_CITY'})
    .drop(columns=['IATA'])
).head()

## Questions

### 1. How many unique airports are in the dataset?

In [ ]:
print(f'There are {len(flights["ORIGIN"].unique())} distinct airports in the dataset')

In [ ]:
print(f'There are {bc.sql("SELECT COUNT(DISTINCT ORIGIN) AS CNT FROM air_transport")["CNT"][0]} distinct airports in the dataset')

### 2. How many flights were delayed and departed early? What is the distribution?

In [ ]:
print(f'{len(flights[flights["DEP_DELAY"] > 0]):,} flights were delayed and {len(flights[flights["DEP_DELAY"] <= 0]):,} left on time or early')

In [ ]:
### calculate the distribution
n_bins = 100

delays = flights[flights['DEP_DELAY'] >  0]['DEP_DELAY']
ontime = flights[flights['DEP_DELAY'] <= 0]['DEP_DELAY']

In [ ]:
%%time
del_bins = np.array([i * 15 for i in range(0, n_bins)], dtype='float64')
delays_binned = delays.digitize(del_bins)
delays_histogram = delays_binned.groupby().count() / len(delays)
(
    delays_histogram
    .set_index(del_bins[delays_histogram.index.to_array()-1])
    .to_pandas()
    .plot(kind='bar', figsize=(20,9), ylim=[0,1.0], title='Delayed departure distribution')
)

In [ ]:
%%time
ontime_bins = np.array([i * (-1) for i in range(n_bins,0,-1)], dtype='float64')
ontime_binned = ontime.digitize(ontime_bins)
ontime_histogram = ontime_binned.groupby().count() / len(ontime)
(
    ontime_histogram
    .set_index(ontime_bins[ontime_histogram.index.to_array()-1])
    .to_pandas()
    .plot(kind='bar', figsize=(20,9), ylim=[0,1.0], title='Early departure distribution')
)

### 3. What are the top 10 airlines and airports with most delays and at least 1000 flights? What is average delay?

In [ ]:
delays = flights[flights['DEP_DELAY'] >  0][['DEP_DELAY', 'ORIGIN', 'DEST', 'OP_UNIQUE_CARRIER']]
ontime = flights[flights['DEP_DELAY'] <= 0][['DEP_DELAY', 'ORIGIN', 'DEST', 'OP_UNIQUE_CARRIER']]

In [ ]:
bc.create_table('delays', delays)
bc.create_table('ontime', ontime)

#### Most delayed

In [ ]:
%%time
bc.sql('''
    SELECT A.ORIGIN
        , B.Name AS ORIGIN_Airport
        , B.City AS ORIGIN_City
        , B.Country AS ORIGIN_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airports AS B
        ON A.ORIGIN = B.IATA
    GROUP BY A.ORIGIN
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.DEST
        , B.Name AS DEST_Airport
        , B.City AS DEST_City
        , B.Country AS DEST_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airports AS B
        ON A.DEST = B.IATA
    GROUP BY A.DEST
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.OP_UNIQUE_CARRIER AS CARRIER
        , B.Name AS CARRIER_Name
        , B.Country AS CARRIER_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    GROUP BY A.OP_UNIQUE_CARRIER
        , B.Name
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

#### Most punctual

In [ ]:
%%time
bc.sql('''
    SELECT A.ORIGIN
        , B.Name AS ORIGIN_Airport
        , B.City AS ORIGIN_City
        , B.Country AS ORIGIN_Country
        , COUNT(*) AS ONTIME_CNT
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airports AS B
        ON A.ORIGIN = B.IATA
    GROUP BY A.ORIGIN
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.DEST
        , B.Name AS DEST_Airport
        , B.City AS DEST_City
        , B.Country AS DEST_Country
        , COUNT(*) AS ONTIME_CNT
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airports AS B
        ON A.DEST = B.IATA
    GROUP BY A.DEST
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.OP_UNIQUE_CARRIER AS CARRIER
        , B.Name AS CARRIER_Name
        , B.Country AS CARRIER_Country
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    GROUP BY A.OP_UNIQUE_CARRIER
        , B.Name
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

## Dates, strings, oh my...

A common misconception is that GPUs are useful for only numeric computations. However, with RAPIDS and BlazingSQL you can perform operations on dates and strings with ease and at GPU's speeds!

### Flights per month and day of week

Even though we already have columns like `YEAR` or `MONTH`, let's calculate these values ourselves.

In [ ]:
%%time
flights['FL_DATE'] = flights['FL_DATE'].astype('datetime64[ms]')
dated = flights[['FL_DATE', 'OP_UNIQUE_CARRIER']]
dated['YEAR'] = dated['FL_DATE'].dt.year
dated['MONTH'] = dated['FL_DATE'].dt.month
dated['DAY'] = dated['FL_DATE'].dt.day
dated['DOW'] = dated['FL_DATE'].dt.dayofweek

In [ ]:
%%time
(
    dated
    .groupby(['YEAR','MONTH'])
    .agg({'FL_DATE': 'count'})
    .to_pandas()
    .plot(kind='bar', figsize=(12,9), title='Total flights per month')
)

In [ ]:
%%time
(
    dated
    .groupby(['MONTH','DAY', 'DOW'])
    .agg({'FL_DATE': 'count'})
    .reset_index()
    .groupby(['DOW'])
    .agg({'FL_DATE': 'mean'})
    .to_pandas()
    .plot(kind='bar', figsize=(12,9), title='Average number of flights per weekday')
)